In [1]:
from pathlib import Path
from langchain.vectorstores import FAISS
from dataset.parsing_data import load_all_documents
import os
from tqdm import tqdm
import pinecone
from langchain.embeddings import GPT4AllEmbeddings, HuggingFaceEmbeddings
from torch import cuda
# from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

load_dotenv()

raw_dataset_dir = Path("../../data/raw_dataset/html")
split_dataset_dir = Path('../../data/split_dataset/500_50')
embedded_dataset_dir = Path('../../data/embedded_dataset/faiss/local_500')

C:\Users\cubix\PycharmProjects\rag\venv39\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
docs = load_all_documents(split_dataset_dir)

Processing RFCs: 9266file [00:49, 185.44file/s]


In [3]:
# embeddings = GPT4AllEmbeddings()
# global_db = FAISS.from_documents([docs[0]], embeddings)
# 
# for idx, doc in tqdm(enumerate(docs[1:]), desc="Embedding in progress", total=len(docs)):
#     global_db.add_documents([doc])
#     if not idx % 80000:
#         global_db.save_local(embedded_dataset_dir / f'faiss_idx_ckpt_{idx}')
# 
# global_db.save_local(embedded_dataset_dir / f'faiss_idx')

Embedding in progress:  73%|███████▎  | 441666/607285 [34:53:32<13:05:03,  3.52it/s] 


AssertionError: 

-------------------------------------------------

In [4]:
embed_model_id = 'sentence-transformers/msmarco-distilbert-dot-v5' #500
# embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2' #250

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 64}
)

In [5]:
global_db = FAISS.from_documents([docs[0]], embed_model)

batch_size = 1000
counter = 0
for idx in tqdm(range(1, len(docs), batch_size), desc="Embedding in progress"):
    global_db.add_documents(docs[idx:idx+batch_size])
    if not counter % 500:
        global_db.save_local(embedded_dataset_dir / f'faiss_idx_ckpt_{idx}')
    counter += 1

global_db.save_local(embedded_dataset_dir / f'faiss_idx')

Embedding in progress: 100%|██████████| 1328/1328 [3:19:00<00:00,  8.99s/it]  


In [33]:
contents = [doc.page_content for doc in docs]
metadatas = [doc.metadata for doc in docs]
clean_metadatas = []
for d in metadatas:
    # Use a dictionary comprehension to create a new dictionary without key-value pairs where the value is None
    d = {key: value for key, value in d.items() if value is not None}
    d = {key: value for key, value in d.items() if value != 'null'}
    d = {key: value for key, value in d.items() if value != []}
    # Update the original dictionary with the modified one
    clean_metadatas.append(d)

In [34]:
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

In [35]:
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENV')
)

index_name = 'test'
pinecone.delete_index(index_name)
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=384,
        metric='cosine'
    )

In [36]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [38]:
from pinecone import Vector
from tqdm import tqdm

batch_size = 1000
counter = 0
for i in tqdm(range(0, len(contents), batch_size)):
    i_end = min(len(contents), i+batch_size)
    contents_batch = contents[i:i+batch_size]
    embeds = embed_model.embed_documents(contents_batch)
    metadatas_batch = clean_metadatas[i:i+batch_size]
    vectors = []
    for embed, metadata in zip(embeds, metadatas_batch):
        vectors.append({'id': str(counter), 'values': embed, 'metadata': metadata})
        counter += 1
    index.upsert(vectors=vectors)
    # add to Pinecone
    # for embed, metadata in zip(embeds, metadatas_batch):
    #     # Add error handling here if needed
    #     try:
    #         # Send a single vector and metadata
    #         index.upsert(vectors=[{'id': str(counter), 'values': embed, 'metadata': metadata}])
    #     except Exception as e:
    #         # Handle the exception, e.g., print an error message
    #         print(f"Error occurred: {e}")
    #     finally:
    #         counter += 1

  4%|▍         | 102/2677 [06:39<2:48:14,  3.92s/it]


ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'Content-Length': '135', 'date': 'Fri, 10 Nov 2023 17:19:56 GMT', 'x-envoy-upstream-service-time': '42', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"code":9,"message":"Starter index record limit reached. Current number: 101000, records in request: 1000, limit: 100000","details":[]}
